<a href="https://colab.research.google.com/github/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Simplificação de Texto

Nas primeiras abordagens, houve uma tentativa de produzir uma rede neural que tivesse como entrada o *token frequence* e saida outro *token frequence*.
 Apesar disso poder ser considerado um resultado válido para a tentativa de melhorar a classificação de texto, não podemos considerar uma frase como um conjunto de palavras com a propriedade da comutatividade.
  A ordem das palavras acaba importando muito para os humanos.

## importando bugingangas

In [0]:
import numpy as np
import csv
import random
#import pandas as pd
from urllib import request as req
from gensim.models import keyedvectors as kv

## Paper With Code

~~Após um gole de sorte, eu acabo por encontrar essa maravilha~~ chamado de [Paper With Code](https://paperswithcode.com/sota/document-summarization-on-cnn-daily-mail), é um ~~bom~~ compilado de resultados cientificos sobre determinados problemas da computação. O primeiro artigo do link de cima, mostra uma solução com ROGUE-1 de 43.83 para o problema de sumarização de documentos como o *state of the art*.

![paper with code](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/paperwithcode.png?raw=true)
<center>paper with code</center>

No artigo *Text Summarization with Pretrained Encoders* (LIU e LAPATA, 2019), os autores representam o texto como *Bert*, que é gerado por uma rede neural e é um caso especial do *word2vec*.




## Representação do Bert word2vec

*Kyubyong* criou um preset do Bert, um dicionário onde cada palavra é representada por um vetor de 768 valores decimais.
Visto algumas limitações das plataformas *Google Colab* e do *Github*, serão usadas apenas 300 instâncias do CNN, que gerou um dicionário reduzido de apenas 25mb.

![Bert is Evil](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/audio-banner.jpg?raw=true)
<center>Bert is Evil</center>

## Base de dados da Cable News Network

A base de dados pode ser encontrado [nesse github da google](https://github.com/google-research-datasets/sentence-compression), no formato *Json*. Essa base de dados é formada com notícias da CNN com a primeira linha da noticia, com o título, com todos os bigramas possíveis e com informações do TAG. Porém no trabalho apenas será usado a primeira linha como entrada e o título como saída.

![Json](https://github.com/x-channel/Mining-Text-Simplifica-o-de-Texto/blob/master/imagens/Jasonf.jpg?raw=true)
<center>Json</center>

## Mesclar o bert com o json.

Basicamente esse script carrega o vocabulário do dicionário bert, onde a partir da segunda linha, cada linha é uma palavra, seguida de sua representação vetorial com 768 valores.
Eventualmente aparecem algumas palavras estranhas como "##,".
Essa palavra significa somente que a palavra anterior termina com uma virgula.

Depois de carregar o vocabulário, ele começa a ler as noticias do json.
Ele segmenta uma noticia, a transformando em um dicionário do python, procura os textos alvos
### jfk['graph']['sentence'] e jfk['headline'].

Com os textos em mãos, ele cata as palavras de cada texto, criando uma matriz de tamanho 768*2 pela quantidade de linhas do subtitulo da noticia.

Observar que para esse código rodar ele deve estar na seguinte pasta ~~por isso está comentado~~. Com o json nomeado *in.json*, com o bert nomeado *dicionariolongo.vec* e uma pasta vazia chamada de *instancias*. O script vai gerar um *dicionario.csv* e varias noticias dentro da pasta instancias. Infelizmente algumas noticias tem caracters que não permitem seu uso como nome do arquivo, por isso todas as noticias foram nomeadas de *arquivo_n.csv* ~~Outro motivo para você guardar seus dados em um único arquivo json~~.

>O diretório do arquivo.
>>in.json \n
dicionariolongo.vec \n
preprobert.py
>>> instancias


In [2]:
"""
#texto para matriz Bert

#!/usr/bin/python
# -*- coding: UTF-8 -*-

import pandas as pd
import csv

import nltk
import numpy as np

import json
import fileinput

import codecs

import os

dici = {}
fins = {}
dico = {} #dicionario reduzido


#carrega o dicionario bert
with open("dicionariolongo.vec", "r", encoding="utf-8") as d:
    for i in d:
        if len(i) > 200: #isso aqui eh para tirar a primeira linha, altura x largura
            try:
                a = i.split()
                if '##' in a[0]:
                    fins[a[0][2:].lower()] = []
                    for j in a[1:]:
                        fins[a[0][2:].lower()].append(float(j))
                elif (not a[0].lower() in dici) or a[0].islower():
                    dici[a[0].lower()] = []
                    for j in a[1:]:
                        dici[a[0].lower()].append(float(j))
            except ValueError:
                print("problema com uma palavra")



with open("in.json", 'rb') as f:
    data = f.read()

data = str(data)[2:-1]
data = data.replace('\\n}', '\\n}-----')
data = data.split('-----')

blanckl = []
for i in range(768):
    blanckl.append('')

instancias = 0
for p in data[0:-1]:
    entrada = []
    saida = []
    try:
        jfk = json.loads(codecs.decode(p, 'unicode_escape'))
        arquivo = jfk['graph']['sentence'].lower().replace("/","").replace("\\","")
        lg = jfk['graph']['sentence'].lower().split()
        st = jfk['headline'].lower().split()

        for i in lg: #isso deveria ser uma funcao, addlist() #semTempo
            if i in dico: #dicionario reduzido
                entrada.append(dico[i])
            elif i in dici:
                dico[i] = dici[i]
                entrada.append(dico[i])
            else:
                for j, jj in fins.items():
                    if j in i[-len(j):]:
                        i = i[:-len(j)]
                        if i in dico: #assim ela poderia ser chamada aqui
                            entrada.append(dico[i])
                        elif i in dici:
                            dico[i] = dici[i]
                            entrada.append(dico[i])
                        if j in dico:
                            entrada.append(dico[j])
                        else:
                            dico[j] = fins[j]
                            entrada.append(dico[j])
                        break
        for i in st: #isso deveria ser uma funcao, addlist() #semTempo
            if i in dico: #dicionario reduzido
                saida.append(dico[i])
            elif i in dici:
                dico[i] = dici[i]
                saida.append(dico[i])
            else:
                for j, jj in fins.items():
                    if j in i[-len(j):]:
                        i = i[:-len(j)]
                        if i in dico: #assim ela poderia ser chamada aqui
                            saida.append(dico[i])
                        elif i in dici:
                            dico[i] = dici[i]
                            saida.append(dico[i])
                        if j in dico:
                            saida.append(dico[j])
                        else:
                            dico[j] = fins[j]
                            saida.append(dico[j])
                        break
    except ValueError:
        print("Houve um Erro")

        
    sv = os.getcwd() + '\\instancias\\arquivo_%i'%instancias + '.csv'
    instancias += 1
    try:
        with open(sv, 'w', newline = '') as file:
            writer = csv.writer(file)
            tam = 0
            while tam < len(entrada):
                tm = []
                #print(type(tm))
                tm.extend(entrada[tam])
                if tam < len(saida):
                    tm.extend(saida[tam])
                else:
                    tm.extend(blanckl)
                tam += 1
                writer.writerow(tm)
    except ValueError:
        print("erro no json")

with open("dicionario.csv", "w", newline = '') as file:
    writer = csv.writer(file)
    for i, ii in dico.items():
	tm = []
	tm.append(i)
	tm.extend(ii)
        writer.writerow(tm)
"""

'\n#texto para matriz Bert\n\n#!/usr/bin/python\n# -*- coding: UTF-8 -*-\n\nimport pandas as pd\nimport csv\n\nimport nltk\nimport numpy as np\n\nimport json\nimport fileinput\n\nimport codecs\n\nimport os\n\ndici = {}\nfins = {}\ndico = {} #dicionario reduzido\n\n\n#carrega o dicionario bert\nwith open("dicionariolongo.vec", "r", encoding="utf-8") as d:\n    for i in d:\n        if len(i) > 200: #isso aqui eh para tirar a primeira linha, altura x largura\n            try:\n                a = i.split()\n                if \'##\' in a[0]:\n                    fins[a[0][2:].lower()] = []\n                    for j in a[1:]:\n                        fins[a[0][2:].lower()].append(float(j))\n                elif (not a[0].lower() in dici) or a[0].islower():\n                    dici[a[0].lower()] = []\n                    for j in a[1:]:\n                        dici[a[0].lower()].append(float(j))\n            except ValueError:\n                print("problema com uma palavra")\n\n\n\nwit

## Abrindo uma noticia (bert) e convertendo para texto.

In [3]:
# Abrindo o dicionario criado com o outro script
dicurl = 'https://raw.githubusercontent.com/x-channel/Mining-Text-Simplifica-o-de-Texto/master/dataset/dicionario.csv'
#dicloader = req.urlopen(dicurl)
vetores = []
palavras = []

with req.urlopen(dicurl) as f:
  meucsv = f.read().decode('charmap')
  meucsv = meucsv.split('\n')[:-1]
  for i in meucsv:
    j = i.replace('\r','').split(',')
    palavras.append(j[0])
    vetores.append(j[1:])

# colocando o dicionario no gensim.
dicio = kv.Word2VecKeyedVectors(768)
dicio.add(palavras, np.array(vetores).astype(float))

#busca por um vetor parecido
five = dicio.get_vector('five')
friends = dicio.get_vector('friends')

print(dicio.similar_by_vector(five, 1), dicio.similar_by_vector(friends, 1))

#abrindo a noticia
#nn = input("digite um numero entre 0 e 199: ")
nn = 1
noticia = 'https://raw.githubusercontent.com/x-channel/Mining-Text-Simplifica-o-de-Texto/master/dataset/noticias/arquivo_%i.csv'%nn

pastaNoticias = 'https://raw.githubusercontent.com/x-channel/Mining-Text-Simplifica-o-de-Texto/master/dataset/noticias/%s_%i.csv'

def abrirNoticia(urlnoticia):
  sentence = []
  head = []
  with req.urlopen(urlnoticia) as f:
    meucsv = f.read().decode('charmap')
    meucsv = meucsv.split('\n')[:-1]
    for i in meucsv:
      j = i.replace('\r',',').replace(',,', ',0.0,').replace(',,', ',0.0,')[:-1]
      j = j.split(',')
      sentence.append(j[:768])
      head.append(j[768:])
  sentence = np.array(sentence).astype(float)
  head = np.array(head).astype(float)
  return sentence, head

def abrirNoticias(urlfolder, total, nome = 'arquivo'):
  primeiro = []
  cabecalh = []
  for i in range(total):
    par = abrirNoticia(urlfolder%(nome,i))
    primeiro.append(par[0])
    cabecalh.append(par[1])
  return np.array(primeiro),np.array(cabecalh)


def vec2head(matriz, model, li):
  head = []
  for i in matriz:
    j = model.similar_by_vector(i,1)
    if j[0][1] > li:
      head.append(j)
  return head

s, h = abrirNoticia(noticia)

titulo = vec2head(h, dicio, 0.5)
print (titulo)

noticias,cabecalhos = abrirNoticias(pastaNoticias, 200)
print(vec2head(noticias[1], dicio, 0.5))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('five', 1.0)] [('friends', 1.0)]
[[('several', 1.0)], [('school', 1.0)], [('districts', 1.0)], [('hold', 1.0)], [('classes', 1.0)], [('on', 1.0)], [('president', 0.6351367831230164)], [("##'", 0.9999998807907104)], [('day', 1.0000001192092896)], [('to', 1.0)], [('make', 1.0)], [('up', 1.0000001192092896)], [('for', 1.0)], [('days', 1.0)], [('missed', 1.0)]]
[[('several', 1.0)], [('school', 1.0)], [('districts', 1.0)], [('in', 1.0)], [('hampton', 1.0)], [('roads', 1.0)], [('are', 0.9999999403953552)], [('holding', 1.0)], [('classes', 1.0)], [('this', 1.0000001192092896)], [('president', 0.6351367831230164)], [("##'", 0.9999998807907104)], [('day', 1.0000001192092896)], [('to', 1.0)], [('make', 1.0)], [('up', 1.0000001192092896)], [('for', 1.0)], [('days', 1.0)], [('missed', 1.0)], [('because', 1.0)], [('of', 1.0)], [('the', 1.0)], [("##'", 0.696808934211731)]]


## Dividindo a base de dados

Aqui a base de dados é dividida em: Treinamento, validação e teste.

OBS: não encontrei isso implementado nem no scikit learn.

In [4]:
#pading, para tudo ter o mesmo tamanho

maxlen = 0

for i in noticias:
  if i.shape[0] > maxlen:
    maxlen = i.shape[0]

for i in range(len(noticias)):
  pad = np.zeros((maxlen, 768))
  pad[:noticias[i].shape[0],:] = noticias[i]
  noticias[i] = pad
  pad = np.zeros((maxlen, 768))
  pad[:cabecalhos[i].shape[0],:] = cabecalhos[i]
  cabecalhos[i] = pad

print(maxlen)

56


In [5]:
treinamento = []
#validacao = []
teste = []

xt = []
yt = []

xr = []
yr = []

atreino = 0.9
#avalidacao = 0.3

d = []
for i in range(len(noticias)):
  d.append(i)

random.shuffle(d)

# deixar essa linha para validar os parametros dos primeiros testes
d = [117, 135, 181, 2, 129, 167, 65, 183, 107, 104, 158, 111, 69, 194, 8, 101, 21, 35, 31, 188, 106, 196, 148, 198, 67, 60, 102, 82, 16, 88, 119, 61, 11, 115, 113, 56, 169, 98, 64, 40, 49, 162, 36, 127, 157, 66, 164, 180, 41, 138, 62, 34, 72, 178, 27, 189, 121, 154, 96, 14, 133, 145, 97, 43, 199, 51, 25, 163, 155, 47, 70, 150, 12, 30, 123, 195, 32, 55, 18, 176, 171, 68, 175, 120, 110, 59, 141, 6, 23, 44, 103, 151, 125, 130, 79, 73, 173, 1, 58, 165, 118, 46, 39, 191, 10, 74, 166, 24, 147, 131, 190, 20, 156, 26, 22, 187, 182, 75, 63, 52, 9, 132, 87, 5, 144, 192, 42, 142, 90, 85, 143, 13, 153, 174, 122, 139, 184, 128, 19, 50, 161, 172, 168, 83, 48, 71, 185, 53, 126, 4, 29, 86, 15, 7, 92, 45, 197, 76, 134, 37, 54, 152, 57, 84, 112, 3, 28, 93, 0, 109, 136, 177, 77, 170, 100, 146, 137, 179, 80, 33, 17, 124, 89, 193, 38, 160, 78, 95, 140, 114, 159, 81, 186, 99, 108, 105, 116, 94, 149, 91]

for i in range(len(noticias)):
  if i < atreino*len(noticias):
    xt.append(noticias[d[i]])
    yt.append(cabecalhos[d[i]])
  else:
    xr.append(noticias[d[i]])
    yr.append(cabecalhos[d[i]])




xt = np.array(xt)
yt = np.array(yt)

xr = np.array(xr)
yr = np.array(yr)

xt = xt.reshape(xt.shape[0], maxlen, 768, 1)
yt = yt.reshape(yt.shape[0], maxlen, 768, 1)

xr = xr.reshape(xr.shape[0], maxlen, 768, 1)
yr = yr.reshape(yr.shape[0], maxlen, 768, 1)

print(len(xt), len(yt))
print(len(xr), len(xr[1]), len(xr[1][1]))

'''
for i in range(len(noticias)):
  if i < atreino*len(noticias):
    treinamento.append(noticias[d[i]])
  elif i < (atreino+avaliacao)*len(noticias):
    validacao.append(noticias[d[i]])
  else:
    teste.append(noticias[d[i]])'''


180 180
20 56 768


'\nfor i in range(len(noticias)):\n  if i < atreino*len(noticias):\n    treinamento.append(noticias[d[i]])\n  elif i < (atreino+avaliacao)*len(noticias):\n    validacao.append(noticias[d[i]])\n  else:\n    teste.append(noticias[d[i]])'

## Construindo a Rede Neural

In [6]:
#Importando o keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Deconvolution2D, Reshape
import keras

Using TensorFlow backend.


In [7]:
model = Sequential()
#model.add(Conv2D(2, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,768,1)))
con = Conv2D(2, kernel_size=(3,3), activation = "tanh", input_shape=(maxlen,768,1),padding="same")
model.add(con)
print(con.input_shape)
print(con.output_shape)
model.add(Conv2D(4, (2,2), activation = "tanh",padding="same"))
print(model.output_shape)
#model.add(MaxPooling2D(pool_size = (2,2)))
#print(model.output_shape)

print()
model.add(Deconvolution2D(1, (2,2),padding="same"))
print(model.output_shape)

#model.add(Reshape((maxlen,768,1)))
#print(model.output_shape)

#model.add(Conv2DTranspose(1,(2,2),output_shape=(768,), activation="tanh"))
#print(model.output_shape)
#rs = Reshape((maxlen,768,1))
#print(rs.output_shape)
#print(rs.input_shape)


model.compile(optimizer='adam',loss='mse',metrics=["acc"])





(None, 56, 768, 1)
(None, 56, 768, 2)
(None, 56, 768, 4)

(None, 56, 768, 1)



## Treinando a rede

In [8]:
model.fit(xt,yt,epochs=100)




Epoch 1/100





180/180 [==============================] - 7s 40ms/step - loss: 5.0080e-04 - acc: 0.8577
Epoch 2/100
180/180 [==============================] - 0s 763us/step - loss: 4.4486e-04 - acc: 0.8577
Epoch 3/100
180/180 [==============================] - 0s 789us/step - loss: 4.0066e-04 - acc: 0.8577
Epoch 4/100
180/180 [==============================] - 0s 752us/step - loss: 3.6607e-04 - acc: 0.8577
Epoch 5/100
180/180 [==============================] - 0s 733us/step - loss: 3.3802e-04 - acc: 0.8577
Epoch 6/100
180/180 [==============================] - 0s 768us/step - loss: 3.1646e-04 - acc: 0.8577
Epoch 7/100
180/180 [==============================] - 0s 769us/step - loss: 2.9915e-04 - acc: 0.8577
Epoch 8/100
180/180 [==============================] - 0s 860us/step - loss: 2.8524e-04 - acc: 0.8577
Epoch 9/100
180/180 [==============================] - 0s 808us/step - loss: 2.7420e-04 - acc: 0.8577
Epoch 10/100
180/180 [==============================] - 0s 778us/step - los

##Teste da rede

Aqui a rede é testada, sem produzir de fato saidas legíveis, mas apenas mostrando uma acurácia da rede em relação ao vetor bert.

In [25]:
print(xt.shape)

aaaaa = xr[0].reshape(1, 56, 768, 1)
bbbbb = yr[0].reshape(56,768)

ttttt = model.predict(aaaaa)

(180, 56, 768, 1)


In [26]:
ttttt = ttttt.reshape(56,768)
print(ttttt.shape)

asdf = vec2head(ttttt,dicio,0.5)
print(asdf)

fdsa = vec2head(bbbbb,dicio,0.5)
print(fdsa)

(56, 768)
[[('201', 0.6480411291122437)], [("##'", 0.64035964012146)], [("##'", 0.6887937784194946)], [('apr', 0.5055166482925415)], [("##'", 0.6681265830993652)], [('201', 0.593322217464447)], [('`', 0.6478577852249146)], [('2013', 0.5896731019020081)], [('`', 0.6571579575538635)], [('##t', 0.5985698699951172)], [("##'", 0.6559717059135437)], [('announces', 0.5409097671508789)], [('announced', 0.6185086965560913)], [('today', 0.6996063590049744)], [('that', 0.7532937526702881)], [('it', 0.6359387040138245)], [('has', 0.6615070104598999)], [('completed', 0.593213677406311)], [('announced', 0.541196346282959)], [('previously', 0.6193296909332275)], [('announced', 0.6645625829696655)], [('acquisition', 0.6641820669174194)], [('##o', 0.5461447834968567)], [('bel', 0.6513062715530396)], [('##o', 0.6455355882644653)], [('##5', 0.5406655073165894)], [('per', 0.574411928653717)], [('##5', 0.610558807849884)], [('per', 0.6707724332809448)], [('share', 0.6802378296852112)], [('cash', 0.57117581

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Teste de predição

Aqui a rede é rodada como teste. A metrica mais usada para esse teste é o ROUGE-1, que conta quantas palavras da saída do sistema está dentro do texto de referência, depois divide pelo número de palavras no texto de referência.

In [27]:


#rouge-1
def rouge(src, ref):
  acertos = 0
  for i in src:
    if i in ref:
      acertos += 1
  return acertos/float(len(ref))

sistem = ['five', 'plane']
refere = ['five', 'in', 'plane']

si = []
re = []

for i in asdf:
  si.append(i[0])

for i in fdsa:
  re.append(i[0])

print(rouge(sistem, refere))
print(rouge(si,re))



0.6666666666666666
0.0


## Referencias

